# Retail Now

## 1. Librerias

In [27]:
import pandas as pd
import numpy as np
import os

# Configura la visualización completa de DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 2. Carga de datos desde CSV

In [30]:
try:
    df_sales = pd.read_csv("./workspace/sales.csv")
    df_inventories = pd.read_csv("./workspace/inventories.csv")
    df_satisfaction = pd.read_csv("./workspace/satisfaction.csv")
    
    # Validar contenido
    if any(df.empty for df in [df_sales, df_inventories, df_satisfaction]):
        raise SystemExit("❌ Error: Algunos archivos CSV están vacíos")
        
except (FileNotFoundError, pd.errors.EmptyDataError) as e:
    raise SystemExit(f"❌ Error crítico: No se pudo cargar los datos: {e}")

# Continuar con análisis



## 3. Limpieza de datos

In [31]:
print("🧹 Limpiando datos...")

# Eliminar filas con valores nulos en cada DataFrame
df_sales_clean = df_sales.dropna().copy()
df_inventories_clean = df_inventories.dropna().copy()
df_satisfaction_clean = df_satisfaction.dropna().copy()

print(f"📊 Ventas después de limpieza: {len(df_sales_clean)} registros")
print(f"📦 Inventarios después de limpieza: {len(df_inventories_clean)} registros")
print(f"😊 Satisfacción después de limpieza: {len(df_satisfaction_clean)} registros")

🧹 Limpiando datos...
📊 Ventas después de limpieza: 10 registros
📦 Inventarios después de limpieza: 10 registros
😊 Satisfacción después de limpieza: 5 registros


## 4. Cálculo de ingresos totales

In [32]:
df_sales_clean['Venta_Total'] = df_sales_clean['Cantidad_Vendida'] * df_sales_clean['Precio_Unitario']

# Calcular ventas totales por tienda
sales_by_store = df_sales_clean.groupby(['ID_Tienda']).agg({
    'Cantidad_Vendida': 'sum',
    'Venta_Total': 'sum'
}).reset_index()

sales_by_store = sales_by_store.rename(columns={
    'Cantidad_Vendida': 'Cantidad_Vendida_Total',
    'Venta_Total': 'Ventas_Totales_tienda'
})

print("📈 Ventas Totales por Tienda:")
print(sales_by_store)

# Resumen estadístico de ventas
print("\n=== RESUMEN ESTADÍSTICO DE VENTAS ===")
print(df_sales_clean['Venta_Total'].describe())

📈 Ventas Totales por Tienda:
   ID_Tienda  Cantidad_Vendida_Total  Ventas_Totales_tienda
0          1                      35                   5000
1          2                      55                  10500
2          3                      50                   9000
3          4                      60                  13000
4          5                      50                  13000

=== RESUMEN ESTADÍSTICO DE VENTAS ===
count       10.000000
mean      5050.000000
std       3361.960407
min       1000.000000
25%       2625.000000
50%       3500.000000
75%       7875.000000
max      10500.000000
Name: Venta_Total, dtype: float64


## 5. Análisis de inventarios (Pandas)

In [33]:
print("📦 Análisis de Inventarios")

# Calcular ventas totales por producto y tienda para comparar con inventario
sales_by_product_store = df_sales_clean.groupby(['ID_Tienda', 'Producto']).agg({
    'Cantidad_Vendida': 'sum'
}).reset_index()

# Unir inventarios con ventas
df_inventories_analysis = pd.merge(
    df_inventories_clean,
    sales_by_product_store,
    on=['ID_Tienda', 'Producto'],
    how='left'
)

# Reemplazar NaN - productos sin ventas - con 0
df_inventories_analysis['Cantidad_Vendida'] = df_inventories_analysis['Cantidad_Vendida'].fillna(0)

# Calcular rotación de inventario (ventas / stock disponible)
df_inventories_analysis['Inventory_Turnover'] = df_inventories_analysis['Cantidad_Vendida'] / df_inventories_analysis['Stock_Disponible']

# Calcular porcentaje de ventas respecto al inventario
df_inventories_analysis['Sales_to_Stock_Ratio'] = (df_inventories_analysis['Cantidad_Vendida'] / df_inventories_analysis['Stock_Disponible']) * 100

print("📊 Rotación de Inventarios por Producto:")
print(df_inventories_analysis[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Cantidad_Vendida', 'Sales_to_Stock_Ratio', 'Inventory_Turnover']])

# Identificar tiendas con inventarios críticos (< 10% de ventas respecto a stock)
critical_inventory = df_inventories_analysis[df_inventories_analysis['Sales_to_Stock_Ratio'] < 10]

print("\n🚨 TIENDAS CON INVENTARIOS CRÍTICOS (<10% ventas/stock):")
if not critical_inventory.empty:
    print(critical_inventory[['ID_Tienda', 'Producto', 'Sales_to_Stock_Ratio']])
else:
    print("No hay tiendas con inventarios críticos en este momento.")

📦 Análisis de Inventarios
📊 Rotación de Inventarios por Producto:
   ID_Tienda    Producto  Stock_Disponible  Cantidad_Vendida  \
0          1  Producto A                50                20   
1          1  Producto B                40                15   
2          2  Producto A                60                30   
3          2  Producto C                45                25   
4          3  Producto A                30                10   
5          3  Producto B                80                40   
6          4  Producto C                70                35   
7          4  Producto A                50                25   
8          5  Producto B                40                20   
9          5  Producto C                60                30   

   Sales_to_Stock_Ratio  Inventory_Turnover  
0             40.000000            0.400000  
1             37.500000            0.375000  
2             50.000000            0.500000  
3             55.555556            0.555556  

## 6. Análisis de satisfacción del cliente (Pandas)

In [34]:
# Combinar satisfacción con ventas totales por tienda
store_performance = pd.merge(
    sales_by_store,
    df_satisfaction_clean,
    on=['ID_Tienda'],
    how='inner'
)

# Filtrar tiendas con baja satisfacción (< 60%)
low_satisfaction_stores = store_performance[store_performance['Satisfacción_Promedio'] < 60]

print("\n⚠️ TIENDAS CON BAJA SATISFACCIÓN (<60%):")
if not low_satisfaction_stores.empty:
    print(low_satisfaction_stores[['ID_Tienda', 'Satisfacción_Promedio', 'Ventas_Totales_tienda']])
else:
    print("Todas las tiendas tienen una satisfacción aceptable.")


⚠️ TIENDAS CON BAJA SATISFACCIÓN (<60%):
   ID_Tienda  Satisfacción_Promedio  Ventas_Totales_tienda
4          5                     55                  13000


## 7. Operaciones (Numpy)

In [35]:
total_revenues_array = store_performance['Ventas_Totales_tienda'].to_numpy()

# Calcular mediana con Numpy usando np.median()
median_revenue = np.median(total_revenues_array)
print(f"\n📈 Mediana de Ingresos Totales: €{median_revenue:,.2f}")

# Calcular desviación estándar con Numpy usando np.std()
std_revenue = np.std(total_revenues_array)
print(f"📉 Desviación Estándar de Ingresos: €{std_revenue:,.2f}")

# Calcular percentiles adicionales con np.percentile()
percentile_25 = np.percentile(total_revenues_array, 25)
percentile_75 = np.percentile(total_revenues_array, 75)

print(f"📊 Percentil 25: €{percentile_25:,.2f}")
print(f"📊 Percentil 75: €{percentile_75:,.2f}")

# Calcular otros estadísticos útiles
mean_revenue = np.mean(total_revenues_array)
min_revenue = np.min(total_revenues_array)
max_revenue = np.max(total_revenues_array)

print(f"📊 Media: €{mean_revenue:,.2f}")
print(f"📊 Mínimo: €{min_revenue:,.2f}")
print(f"📊 Máximo: €{max_revenue:,.2f}")


📈 Mediana de Ingresos Totales: €10,500.00
📉 Desviación Estándar de Ingresos: €2,973.21
📊 Percentil 25: €9,000.00
📊 Percentil 75: €13,000.00
📊 Media: €10,100.00
📊 Mínimo: €5,000.00
📊 Máximo: €13,000.00


## 8. Simulacion y proyección futura (Numpy)

In [36]:
np.random.seed(2026)

# Parámetros basados en datos actuales
mean_revenue = np.mean(total_revenues_array)
std_revenue = np.std(total_revenues_array)

print(f"📊 Parámetros de simulación:")
print(f"   Media actual: €{mean_revenue:,.2f}")
print(f"   Desviación estándar actual: €{std_revenue:,.2f}")

# Simular 12 meses de proyecciones para cada tienda
n_months = 12
n_stores = len(store_performance)

# Generar arrays aleatorios usando numpy.random.normal() 
# siguiendo una distribución normal
projections = np.random.normal(
    loc=mean_revenue,      # Media de la distribución
    scale=std_revenue,     # Desviación estándar de la distribución
    size=(n_stores, n_months)  # Forma del array: (tiendas, meses)
)

# Asegurar que no haya valores negativos (las ventas no pueden ser negativas)
# usando np.abs()
projections = np.abs(projections)

print(f"\n📊 Array de proyecciones generado:")
print(f"   Forma: {projections.shape}")
print(f"   Tipo: {projections.dtype}")
print(f"   Rango: €{projections.min():,.2f} - €{projections.max():,.2f}")

# Crear DataFrame con proyecciones
months = [f"Mes_{i+1}" for i in range(n_months)]
df_projections = pd.DataFrame(
    projections,
    columns=months,
    index=store_performance['ID_Tienda']
)

print("\n📈 PROYECCIONES DE VENTAS (12 meses):")
print(df_projections.round(0))

# Calcular estadísticas de proyecciones
print("\n📊 Resumen de Proyecciones:")
print(f"Ingreso promedio proyectado mensual: €{np.mean(projections):,.2f}")
print(f"Mejor escenario (max): €{np.max(projections):,.2f}")
print(f"Peor escenario (min): €{np.min(projections):,.2f}")
print(f"Desviación estándar de proyecciones: €{np.std(projections):,.2f}")

📊 Parámetros de simulación:
   Media actual: €10,100.00
   Desviación estándar actual: €2,973.21

📊 Array de proyecciones generado:
   Forma: (5, 12)
   Tipo: float64
   Rango: €370.09 - €18,216.13

📈 PROYECCIONES DE VENTAS (12 meses):
             Mes_1    Mes_2    Mes_3    Mes_4    Mes_5    Mes_6    Mes_7  \
ID_Tienda                                                                  
1           8816.0   5959.0  11026.0  10061.0  14410.0  10986.0   7633.0   
2           4154.0  12800.0  11536.0  10760.0  12074.0   6396.0  15954.0   
3          10267.0   5048.0   9026.0  18216.0   9390.0   5866.0   9610.0   
4          10083.0   9895.0  10535.0   8811.0  10070.0  10506.0  14491.0   
5          10849.0   9618.0   8678.0   7128.0   3523.0  11807.0    370.0   

             Mes_8    Mes_9   Mes_10   Mes_11   Mes_12  
ID_Tienda                                               
1           5354.0  11924.0   9156.0   9880.0   8290.0  
2           9153.0  11170.0  12319.0   1998.0  15081.0  
3  